In [ ]:
!aws s3api list-objects --bucket usgs-landsat --prefix collection02/level-2/standard/oli-tirs/2021/143/054 --request-payer requester > scans.json

In [ ]:
import json
import os

def load_json(file_name):
    with open(file_name) as jsonFile:
        scans = json.load(jsonFile)
        jsonFile.close()
    return scans


def get_s3_locations():
    scans = load_json("scans.json")
    
    locations = []

    for item in scans["Contents"]:
        key = item["Key"]

        if("thumb_large.jpeg" in key):
            loc = {}
            key_split = key.split("/")
            loc["folder_name"] = key_split[7]
            loc["s3_path"] = "/".join(key_split[0:8])
            locations.append(loc)
            
    return locations
    
def generate_s3_copy_commands(locations):
    base_command = "aws s3 cp --recursive --request-payer requester s3://usgs-landsat/"
    destination_path = "./landsat_data/"
    
    commands = ""
    for item in locations:
        command = base_command + item["s3_path"] + "/ " + destination_path + item["folder_name"] + "/"
        commands = commands + "\n" + command
        

    f = open("copy_commands.sh", "a")
    f.write(commands)
    f.close()
        
    return commands
        

In [ ]:
locations = get_s3_locations()
commands = generate_s3_copy_commands(locations)
commands

In [ ]:
!sh copy_commands.sh